In [ ]:
!pip install umap-learn
!pip install adversarial-robustness-toolbox

In [ ]:
import pandas as pd
import numpy as np
import umap
import tensorflow as tf

from umap import UMAP
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from art.attacks.evasion import FastGradientMethod, CarliniLInfMethod
from tensorflow.keras.datasets import mnist, cifar10
from art.estimators.classification import KerasClassifier
tf.compat.v1.disable_eager_execution()

In [ ]:
# Load the MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Load CIFAR10 dataset
#(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Flatten and normalize the data
x_train = x_train.reshape((x_train.shape[0], -1)) / 255.0
x_test = x_test.reshape((x_test.shape[0], -1)) / 255.0

# Create a copy of the original data for legitimate samples
legitimate_x = np.copy(x_test)
legitimate_y = np.copy(y_test)


In [ ]:
# Build UMAP 2D, 3D, 5D and 10D components
umap_2d = UMAP(n_neighbors=15, n_components=2, min_dist=0.1, random_state=42)
umap_3d = UMAP(n_neighbors=15, n_components=3, min_dist=0.1, random_state=42)
umap_5d = UMAP(n_neighbors=15, n_components=5, min_dist=0.1, random_state=42)
umap_10d = UMAP(n_neighbors=15, n_components=10, min_dist=0.1, random_state=42)

# Transform legitimate data to 2D, 3D, 5D and 10D
umap_2d_legitimate = umap_2d.fit_transform(legitimate_x)
umap_3d_legitimate = umap_3d.fit_transform(legitimate_x)
umap_5d_legitimate = umap_2d.fit_transform(legitimate_x)
umap_10d_legitimate = umap_3d.fit_transform(legitimate_x)

In [ ]:
# Load pre-trained model
model = tf.keras.models.load_model("mnist_CNN_model.h5") # MNIST

#model = tf.keras.models.load_model("cifar10_CNN_model.h5") # CIFAr10

In [ ]:
# Create an ART classifier for FGSM attack
classifier_fgsm = KerasClassifier(model=model, clip_values=(0, 1), use_logits=False)

# Generate adversarial examples for FGSM
legitimate_x = legitimate_x.reshape(-1, 28, 28, 1) #MNIST
#legitimate_x = legitimate_x.reshape(-1, 32, 32, 3) #CIFAR10
fgsm = FastGradientMethod(classifier_fgsm, eps=0.01)
x_test_adv_fgsm = fgsm.generate(legitimate_x)

# Transform the adversarial data to 2D and 3D
x_test_adv_fgsm = x_test_adv_fgsm.reshape(x_test_adv_fgsm.shape[0], -1)
umap_2d_attack_fgsm = umap_2d.fit_transform(x_test_adv_fgsm)
umap_3d_attack_fgsm = umap_3d.fit_transform(x_test_adv_fgsm)
umap_5d_attack_fgsm = umap_5d.fit_transform(x_test_adv_fgsm)
umap_10d_attack_fgsm = umap_10d.fit_transform(x_test_adv_fgsm)


In [ ]:
# Create UMAP transformers for 2D and 3D
column_names = [f"umap{i}" for i in range(2)]
data_2d_legitimate = pd.DataFrame(umap_2d_legitimate, columns=column_names)
data_2d_attack_fgsm = pd.DataFrame(umap_2d_attack_fgsm, columns=column_names)

column_names = [f"umap{i}" for i in range(3)]
data_3d_legitimate = pd.DataFrame(umap_3d_legitimate, columns=column_names)
data_3d_attack_fgsm = pd.DataFrame(umap_3d_attack_fgsm, columns=column_names)

column_names = [f"umap{i}" for i in range(5)]
data_5d_legitimate = pd.DataFrame(umap_5d_legitimate, columns=column_names)
data_5d_attack_fgsm = pd.DataFrame(umap_5d_attack_fgsm, columns=column_names)

column_names = [f"umap{i}" for i in range(10)]
data_10d_legitimate = pd.DataFrame(umap_10d_legitimate, columns=column_names)
data_10d_attack_fgsm = pd.DataFrame(umap_10d_attack_fgsm, columns=column_names)

In [ ]:
# Add labels for the features
data_2d_legitimate["attack"] = 0  # Legitimate samples labeled as 0
data_2d_attack_fgsm["attack"] = 1  # FGSM attack samples labeled as 1

data_3d_legitimate["attack"] = 0  
data_3d_attack_fgsm["attack"] = 1  

data_5d_legitimate["attack"] = 0  
data_5d_attack_fgsm["attack"] = 1  

data_10d_legitimate["attack"] = 0  
data_10d_attack_fgsm["attack"] = 1  

# Combine the dataframes
UMAP2D_data = pd.concat([data_2d_legitimate, data_2d_attack_fgsm], axis=0)
UMAP3D_data = pd.concat([data_3d_legitimate, data_3d_attack_fgsm], axis=0)
UMAP5D_data = pd.concat([data_5d_legitimate, data_5d_attack_fgsm], axis=0)
UMAP10D_data = pd.concat([data_10d_legitimate, data_10d_attack_fgsm], axis=0)

# Save the dataframe to a CSV file
UMAP2D_data.to_csv("Umap_data_2D_fgsm.csv", index=False)
UMAP3D_data.to_csv("Umap_data_3D_fgsm.csv", index=False)
UMAP5D_data.to_csv("Umap_data_5D_fgsm.csv", index=False)
UMAP10D_data.to_csv("Umap_data_10D_fgsm.csv", index=False)

### Carlini-Wagner

In [ ]:
# Create an ART classifier for CW attack
classifier_cw = KerasClassifier(model=model, clip_values=(0, 1), use_logits=False)

# Generate adversarial examples for CW
Adv_xcw = legitimate_x.reshape(-1, 28, 28, 1) #MNIST
#Adv_xcw = legitimate_x.reshape(-1, 32, 32, 3) #CIFAR10

Adv_cw = CarliniLInfMethod(classifier_cw, confidence=1.0, max_iter=20, targeted=False, initial_const=1e-3, largest_const=2e-3)
x_test_adv_cw = Adv_cw.generate(Adv_xcw)

In [ ]:
# Transform the adversarial data to 2D using UMAP
Adv_xcw = Adv_xcw.reshape(Adv_xcw.shape[0], -1)
umap_2d_attack_cw = umap_2d.fit_transform(Adv_xcw)
umap_3d_attack_cw = umap_3d.fit_transform(Adv_xcw)
umap_5d_attack_cw = umap_5d.fit_transform(Adv_xcw)
umap_10d_attack_cw = umap_10d.fit_transform(Adv_xcw)

In [ ]:
# Create UMAP transformers for 2D and 3D
column_names = [f"umap{i}" for i in range(2)]
data_2d_legitimate = pd.DataFrame(umap_2d_legitimate, columns=column_names)
data_2d_attack_cw = pd.DataFrame(umap_2d_attack_cw, columns=column_names)

column_names = [f"umap{i}" for i in range(3)]
data_3d_legitimate = pd.DataFrame(umap_3d_legitimate, columns=column_names)
data_3d_attack_cw = pd.DataFrame(umap_3d_attack_cw, columns=column_names)

column_names = [f"umap{i}" for i in range(5)]
data_5d_legitimate = pd.DataFrame(umap_5d_legitimate, columns=column_names)
data_5d_attack_cw = pd.DataFrame(umap_5d_attack_cw, columns=column_names)

column_names = [f"umap{i}" for i in range(10)]
data_10d_legitimate = pd.DataFrame(umap_10d_legitimate, columns=column_names)
data_10d_attack_cw = pd.DataFrame(umap_10d_attack_cw, columns=column_names)

In [ ]:
# Add labels for the features
data_2d_legitimate["attack"] = 0  # Legitimate samples labeled as 0
data_2d_attack_cw["attack"] = 1  # CW attack samples labeled as 1

data_3d_legitimate["attack"] = 0  
data_3d_attack_cw["attack"] = 1  

data_5d_legitimate["attack"] = 0  
data_5d_attack_cw["attack"] = 1

data_10d_legitimate["attack"] = 0  
data_10d_attack_cw["attack"] = 1

# Combine the dataframes
UMAP2D_data = pd.concat([data_2d_legitimate, data_2d_attack_cw], axis=0)
UMAP3D_data = pd.concat([data_3d_legitimate, data_3d_attack_cw], axis=0)
UMAP5D_data = pd.concat([data_5d_legitimate, data_5d_attack_cw], axis=0)
UMAP10D_data = pd.concat([data_10d_legitimate, data_10d_attack_cw], axis=0)

# Save the dataframe to a CSV file
UMAP2D_data.to_csv("Umap_data_2D_cw.csv", index=False)
UMAP3D_data.to_csv("Umap_data_3D_cw.csv", index=False)
UMAP5D_data.to_csv("Umap_data_5D_cw.csv", index=False)
UMAP10D_data.to_csv("Umap_data_5D_cw.csv", index=False)